## Install the required packages

In [22]:
!pip install gradio assemblyai sentence-transformers chromadb

## Import the required libraries.

In [23]:
import gradio as gr
import librosa
import numpy as np
import matplotlib.pyplot as plt
import base64
import json
from io import BytesIO
from IPython.display import Audio
import assemblyai as aai
from sentence_transformers import SentenceTransformer
import chromadb

In [ ]:
# AssemblyAI API Key
aai.settings.api_key = "YOUR_ASSEMBLY_AI_API"

# Path to the chroma-db embeddings
path = r'/content/drive/MyDrive/search_engine/db/'

# ChromaDB Setup
client = chromadb.PersistentClient(path=path)
collection = client.get_collection(name="subtitle_chunks")

print(client.heartbeat())

1742714477312490073


In [25]:
# Transcribe Audio Function
def transcribe_audio(audio_file):
    if audio_file is None:
        return "Please upload an audio file.", None  # Return empty text

    config = aai.TranscriptionConfig(language_code="en")
    transcriber = aai.Transcriber(config=config)
    transcript = transcriber.transcribe(audio_file)

    return transcript.text, transcript.text  # Return text & store in state

# Format Subtitle Search Results
def format_results_as_json(results):
    formatted_results = []
    for i, metadata in enumerate(results["metadatas"][0]):
        subtitle_name = metadata["subtitle_name"]
        subtitle_id = metadata["subtitle_id"]
        url = f"https://www.opensubtitles.org/en/subtitles/{subtitle_id}"
        formatted_results.append({
            "Result": i + 1,
            "Subtitle Name": subtitle_name.upper(),
            "URL": url,
        })
    return json.dumps(formatted_results, indent=4)

# Subtitle Search Function
def retrieve_and_display_results(query):
    if not query:
        return "No transcription text available for search."

    model = SentenceTransformer("all-MiniLM-L6-v2")
    query_embedding = model.encode([query], show_progress_bar=False).tolist()
    results = collection.query(query_embeddings=query_embedding, n_results=5, include=["metadatas"])

    return format_results_as_json(results)

# Clear Function
def clear_all():
    return "", ""

# Gradio Interface with Styling
custom_css = """
    .gradio-container { width: 90% !important; margin: auto; }
    button { padding: 10px; border-radius: 5px; cursor: pointer; transition: opacity 0.3s; }
    .transcribe-btn { background-color: green; color: white; }
    .transcribe-btn:hover { opacity: 0.8; }
    .search-btn { background-color: blue; color: white; }
    .search-btn:hover { opacity: 0.8; }
    .clear-btn { background-color: #ff4500; color: white; }
    .clear-btn:hover { opacity: 0.8; }
"""

with gr.Blocks(css=custom_css) as demo:
    gr.Markdown("# 🎵 Shazam Clone : Audio Transcription & Subtitle Search", elem_id="title")
    text_state = gr.State(value="")  # Store transcribed text

    with gr.Row():
        audio_input = gr.Audio(type="filepath", label="Upload Audio")
        transcribed_text = gr.Textbox(label="Transcribed Text", interactive=False)

    with gr.Row():
        transcribe_button = gr.Button("Transcribe", elem_classes=["transcribe-btn"])
        search_button = gr.Button("Search Subtitles",elem_classes=["search-btn"] )
        clear_button = gr.Button("Clear", elem_classes=["clear-btn"])

    search_results = gr.Textbox(label="Subtitle Search Results")

    # Button Click Actions
    transcribe_button.click(transcribe_audio, inputs=[audio_input], outputs=[transcribed_text, text_state])
    search_button.click(retrieve_and_display_results, inputs=[text_state], outputs=[search_results])
    clear_button.click(clear_all, inputs=[], outputs=[transcribed_text, search_results])

# Launch App
demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://0789dd5d92321ee4c6.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
